In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.model_fpgan import FPGANSynthesizer
from models.model_ctgan import CTGANSynthesizer
from models.model_dpctgan import DPCTGANSynthesizer

import ctgan
from sklearn.model_selection import train_test_split
from utils.privacy import normalized_avg_dist
from utils.fidelity import get_predictions, eval_fidelity, get_accuracy, get_roc_auc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("D:/Datasets/fetal_health/fetal_health.csv")

data.fetal_health = data.fetal_health.replace(1, 0)
data.fetal_health = data.fetal_health.replace(2, 1)
data.fetal_health = data.fetal_health.replace(3, 1)

data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,1.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,0.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,0.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,0.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,0.0


In [4]:
data.shape

(2126, 22)

In [5]:
#normalized_data=(data-data.mean())/data.std()
data=(data-data.min())/(data.max()-data.min())

In [6]:
target = 'fetal_health'

In [7]:
X = data.drop(columns=[target])
y = data[target]

In [10]:
#model_ctgan = CTGANSynthesizer(epochs=100, verbose=True)
#model_ctgan = DPCTGANSynthesizer(verbose=True)
model_ctgan = FPGANSynthesizer(epochs=100, verbose=True)
#original_losses, tranformed_privacies, losses = model_ctgan.fit(data)
model_ctgan.fit(data)

Epoch 1, Loss G: -0.1957,Loss D: -0.0727
Epoch 2, Loss G: -0.1893,Loss D: -0.2602
Epoch 3, Loss G: -0.1834,Loss D: -0.5246
Epoch 4, Loss G: -0.2166,Loss D: -0.7162
Epoch 5, Loss G: -0.2612,Loss D: -0.7350
Epoch 6, Loss G: -0.3056,Loss D: -0.6776
Epoch 7, Loss G: -0.3423,Loss D: -0.3874
Epoch 8, Loss G: -0.3300,Loss D: -0.2563
Epoch 9, Loss G: -0.3445,Loss D: -0.0546
Epoch 10, Loss G: -0.3748,Loss D:  0.3383
Epoch 11, Loss G: -0.3801,Loss D:  0.5534
Epoch 12, Loss G: -0.3662,Loss D:  0.6221
Epoch 13, Loss G: -0.3235,Loss D:  0.2617
Epoch 14, Loss G: -0.2694,Loss D: -0.1064
Epoch 15, Loss G: -0.2249,Loss D: -0.3578
Epoch 16, Loss G: -0.1967,Loss D: -0.4783
Epoch 17, Loss G: -0.1633,Loss D: -0.6149
Epoch 18, Loss G: -0.1538,Loss D: -0.5628
Epoch 19, Loss G: -0.1835,Loss D: -0.4231
Epoch 20, Loss G: -0.2208,Loss D: -0.0442
Epoch 21, Loss G: -0.2272,Loss D:  0.1639
Epoch 22, Loss G: -0.1905,Loss D: -0.2052
Epoch 23, Loss G: -0.1365,Loss D: -0.6344
Epoch 24, Loss G: -0.0677,Loss D: -0.6681
E

([array(-0.00148512, dtype=float32),
  array(0.00046595, dtype=float32),
  array(-0.00089122, dtype=float32),
  array(0.00637911, dtype=float32),
  array(0.00652431, dtype=float32),
  array(0.01350469, dtype=float32),
  array(0.0164557, dtype=float32),
  array(0.0223238, dtype=float32),
  array(0.02260456, dtype=float32),
  array(0.02769537, dtype=float32),
  array(0.01631084, dtype=float32),
  array(0.0304893, dtype=float32),
  array(0.00394282, dtype=float32),
  array(-0.00899005, dtype=float32),
  array(-0.02088175, dtype=float32),
  array(-0.0419326, dtype=float32),
  array(-0.08116085, dtype=float32),
  array(-0.1016686, dtype=float32),
  array(-0.11146063, dtype=float32),
  array(-0.13778268, dtype=float32),
  array(-0.17987007, dtype=float32),
  array(-0.20547734, dtype=float32),
  array(-0.21403201, dtype=float32),
  array(-0.232801, dtype=float32),
  array(-0.2822305, dtype=float32),
  array(-0.2809331, dtype=float32),
  array(-0.29157653, dtype=float32),
  array(-0.31643483, 

In [11]:
from utils.utils import save_models
save_models(model_ctgan, 'fpgan100', 'fetal_health')

In [ ]:
from utils.utils import load_models
model_ctgan = load_models('dpctgan3', 'fetal_health')

In [ ]:
samples = model_ctgan.sample(len(data))  # Synthetic copy
samples.dropna(how='any', inplace=True)

X_syn = samples.drop(columns=[target])
y_syn = samples[target]
y_syn = y_syn.round(0)
y_syn = y_syn.astype(int)

## Privacy

In [ ]:
samples = samples.apply(pd.to_numeric) # convert all columns of DataFrame
data = data.apply(pd.to_numeric) # convert all columns of DataFrame
samples = samples[:2000]
data = data[:2000]

In [ ]:
normalized_avg_distance = normalized_avg_dist(samples, data)
print(normalized_avg_distance)

## Fidelity

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

X_syn_train, X_syn_test, y_syn_train, y_syn_test = train_test_split(X_syn, y_syn, test_size=0.3, random_state=42, stratify=y_syn)

rr_pred = []
fr_pred = []
ff_pred = []
rf_pred = []

rr_pred = get_predictions(X_train, y_train, X_test, y_test)
fr_pred = get_predictions(X_syn_train, y_syn_train, X_test, y_test)

In [ ]:
from sklearn import metrics
def print_confusion_matrix(rr_pred, fr_pred):
    confusion_matrix = metrics.confusion_matrix(rr_pred, fr_pred)

    cm_display1 = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

    cm_display1.plot()
    plt.show()
    
print_confusion_matrix(rr_pred, fr_pred)

In [ ]:
percentage = eval_fidelity(rr_pred, fr_pred)

In [ ]:
rr_acc = get_accuracy(y_test, rr_pred)
fr_acc = get_accuracy(y_test, fr_pred)
print("Accuracy of real data: ", rr_acc)
print("Accuracy of synthetic data: ", fr_acc)

In [ ]:
rr_score = get_roc_auc(y_test, rr_pred)
fr_score = get_roc_auc(y_test, fr_pred)
print("ROC AUC of real data: ", rr_score)
print("ROC AUC of synthetic data: ", fr_score)

In [ ]:
#print(class1_ratio, class2_ratio)

## Final plot

In [ ]:
plt.scatter(percentage, normalized_avg_distance, c ="blue")
plt.xlabel("fidelity")
plt.ylabel("privacy")
plt.show()
print(percentage, normalized_avg_distance)